In [1]:
!git clone https://github.com/TheBrainCodeGames/2021_hackaton_material
!mv 2021_hackaton_material/bcg_auxiliary.py ./bcg_auxiliary.py
!mv 2021_hackaton_material/bz_LoadBinary.py ./bz_LoadBinary.py
!rm -r 2021_hackaton_material

Cloning into '2021_hackaton_material'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 21 (delta 6), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [2]:
from bcg_auxiliary import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import RobustScaler

# Importar datos

In [3]:
!git clone https://github.com/cognoma/figshare.git
%cd /content/figshare
!python setup.py install

Cloning into 'figshare'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 53 (delta 6), reused 9 (delta 1), pack-reused 35
Unpacking objects: 100% (53/53), done.
/content/figshare
running install
running bdist_egg
running egg_info
creating figshare.egg-info
writing figshare.egg-info/PKG-INFO
writing dependency_links to figshare.egg-info/dependency_links.txt
writing requirements to figshare.egg-info/requires.txt
writing top-level names to figshare.egg-info/top_level.txt
writing manifest file 'figshare.egg-info/SOURCES.txt'
adding license file 'LICENSE.md'
writing manifest file 'figshare.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/figshare
copying figshare/figshare.py -> build/lib/figshare
copying figshare/__init__.py -> build/lib/figshare
creating build/bdist.linux-

In [4]:
from figshare.figshare import Figshare
fs = Figshare()
fs.retrieve_files_from_article(16847521, directory="/content/Datos/Entrenamiento/Amigo2")
fs.retrieve_files_from_article(16856137, directory="/content/Datos/Entrenamiento/Som2")
fs.retrieve_files_from_article(14959449, directory="/content/Datos/Validacion/Dlx1")
fs.retrieve_files_from_article(14960085, directory="/content/Datos/Validacion/Thy7")
fs.retrieve_files_from_article(16856182, directory="/content/Datos/Test/Test1")
fs.retrieve_files_from_article(16856248, directory="/content/Datos/Test/Test2")

# Funciones

## Función getPendientes

In [5]:
def getPendientes(canal):
	"""
	Obtiene las diferencias de amplitud entre un punto y su vecino anterior y posterior

	Parameters
	----------
	canal : Numpy array (N x 1)
		Numpy array que contiene las amplitudes de un canal

	Returns
	-------
	diffs : Numpy array (N-1 x 1)
		Numpy array  que contiene las pendientes de los puntos repsecto a sus dos vecinos

	Example
	-------
	diffs = getPendientes(canal0)
	"""

	diffs = list()
	for i in range(0, len(canal) - 1):
		diffs.append(canal[i+1] - canal[i])
	return np.array(diffs)

## Función getCharacteristics

In [6]:
def getCharacteristics(differences, amplitudes, intervalo=400):
	"""
	Obtiene las caracteristicas de un ripple

	Parameters
	----------
	differences : Numpy Array (N x 1)
		Numpy array  que contiene las pendientes de los puntos 
		respecto a sus dos vecinos
	amplitudes :  Numpy Array (N x 1)
		Numpy array que contiene las amplitudes de un canal
	intervalo : int
		Tamaño de los intervalos en los que se dividira la senal

	Returns
	-------
	sumsPend : Numpy Array (N - intervalo x 1)
		Numpy array  que contiene la suma de las pendientes de los 
		puntos dentro de un intervalo
	sumsAmpl : Numpy Array (N - intervalo x 1)
		Numpy array  que contiene la suma de las amplitudes de los 
		puntos dentro de un intervalo
	sumsAbsAmpl	: Numpy Array (N - intervalo x 1)
		Numpy array con la suma de las amplitudes en valor absoluto
		de los puntos dentro de un intervalo
	Example
	-------
	pendientes, amplitudes, absampl = getCharacteristics(diffs, canal, 200)
	"""

	absolutes = np.abs(differences)
	absamp = np.abs(amplitudes)
	sumsPend = list()
	sumsAmpl = list()
	sumsAbsAmpl = list()

	sumsPendInter = np.sum(absolutes[:intervalo])
	sumsPend.append(sumsPendInter)
	sumsAmplInter = np.sum(amplitudes[:intervalo])
	sumsAmpl.append(sumsAmplInter)
	sumsAbsAmplInter = np.sum(absamp[:intervalo])
	sumsAbsAmpl.append(sumsAbsAmplInter)

	for i in range(1, len(absolutes) - intervalo):
		sumsPendInter = sumsPendInter + absolutes[i+intervalo] - absolutes[i-1]
		sumsPend.append(sumsPendInter)
		sumsAmplInter = sumsAmplInter + amplitudes[i+intervalo] - amplitudes[i-1]
		sumsAmpl.append(sumsAmplInter)
		sumsAbsAmplInter = sumsAbsAmplInter + absamp[i+intervalo] - absamp[i-1]
		sumsAbsAmpl.append(sumsAbsAmplInter)

	return np.array(sumsPend), np.array(sumsAmpl), np.array(sumsAbsAmpl)

## Función solsToIntervals

In [7]:
def solsToIntervals(solutions, intervalo=400, porcentaje=0.5):
  """
	Pasa el array de soluciones a intervalos

	Parameters
	----------
	solutions : Numpy Array (N x 1)
		Numpy array  que contiene si un punto es ripple o no
	intervalo : int
		Tamaño de los intervalos en los que esta dividida la senal
	porcentaje : int
		Porcentaje del numero de 1s que debe haber en un intervalo
		para que se considere ripple
	
	Returns
	-------
	sumsSols : Numpy Array (N - intervalo - 1 x 1)
		Numpy array  que contiene  si un intervalo
		es ripple o no	

	Example
	-------
	solsIntervalo = solsToIntervals(signal)

	"""
  
  sumsSols = list()

  sumsSolsInter = np.sum(solutions[:intervalo])
  sumsSols.append(sumsSolsInter)
  umbral = int(porcentaje * intervalo)
  
  for i in range(1, len(solutions) - intervalo - 1):
    sumsSolsInter = sumsSolsInter + solutions[i+intervalo] - solutions[i-1]
    sol = 1 if sumsSolsInter >= umbral else 0
    sumsSols.append(sol)
  
  return np.array(sumsSols)

## Función getDataSet

In [8]:
def getDataSet(pendientes, amplitudes, abs_amplitudes):
	"""
	Une las caracteristicas del ripple en un mismo array

	Parameters
	----------
	pendientes : Numpy Array (N x 1)
		Numpy array  que contiene la suma de las pendientes de los 
		puntos dentro de un intervalo
	amplitudes : Numpy Array (N x 1)
		Numpy array  que contiene la suma de las amplitudes de los 
		puntos dentro de un intervalo
	abs_amplitudes : Numpy Array (N x 1)
		Numpy array  que contiene la suma de las amplitudes en
		valor absoluto de los puntos dentro de un intervalo

	Returns
	-------
	dataset : Numpy Array (N x 3)
		Numpy array  que contiene los array de 
		pendientes y amplitudes unidos

	Example
	-------
	train = getDataSet(pendientes, amplitudes, absampl)
	"""
	dataset = np.zeros([len(pendientes), 3])
	dataset[:, 0] = pendientes
	dataset[:, 1] = amplitudes
	dataset[:, 2] = abs_amplitudes
	return dataset

## Función toTimes

In [9]:
def toTimes(prediction, fs, zeros=400, intervalo=400) :
  """
  Pasa la prediccion de los intervalos a intervalos completos de tiempo

  Parameters
  ----------
  prediction : Numpy Array (N x 1)
    Numpy array que indica si un intervalo es ripple o no
  fs : int
    Frecuencia de sampleo
  zeros : int
    Numero de ceros que tiene que pasar para que se considere
    que el ripple ha terminado
  intervalo : int
    Tamaño de los intervalos en los que esta dividida la senal

  Returns
  -------
  times : Numpy Array (N x 1)
    Numpy array con las predicciones pasadas a tiempos

  Example
  -------
  tiemposBoostPredict = toTimes(boostPredict, fs, zeros=700, intervalo=400)
  """
  insideRipple = False
  ripples = list()

  start = 0
  end = 0
  zerosCount = 0
  zerosUmbral = zeros

  for i, n in enumerate(prediction):
    if (n == 1):
      if (not insideRipple):
        insideRipple = True
        start = i
      zerosCount = 0
    elif (n == 0 and insideRipple):
      zerosCount += 1
      if (zerosCount >= zerosUmbral):
        end =  i - zerosUmbral
        ripples.append(np.array([start, end]))
        insideRipple = False
        start, end = 0, 0
  
  times = np.array(ripples)
  times[:,1] = times[:,1] + intervalo
  return times / fs

## Undersampling

In [10]:
def undersamplingIndexes(data) :
    """
    Hace un undersampling de los datos

    Parameters
    ----------
    data : Numpy Array (N x 8)
        Numpy array con los datos

    Returns
    -------
    undersamplingIndexes : Numpy Array (N x 1)
        Numpy array con los indices de os datos

    Example
    -------
    indexes = undersamplingIndexes(solsIntervalo)
    """
    ripples_unos = list()
    ripples_zeros = list()

    for i in range(0, len(data) - 1):
        if (data[i]==1):
            ripples_unos.append(i)
        else:
            ripples_zeros.append(i)

    yes_ripple = np.array(ripples_unos)
    non_ripple = np.array(np.random.choice(ripples_zeros, 7000000, replace = False))
    undersamplingIndexes = np.concatenate([non_ripple, yes_ripple])
    undersamplingIndexes = np.sort(undersamplingIndexes)
    return undersamplingIndexes

# Algoritmo

In [11]:
inter = 700
porcen = 0.6
zeros = 700

Obtener los datos de entrenamiento

In [12]:
datapath = "/content/Datos/Entrenamiento/Som2/figshare_16856137"
data_som2, fs_som2, session_name = load_data(datapath)
ripples_tags_som2 = load_ripples_tags(datapath, fs_som2)
signal_som2 = get_ripples_tags_as_signal(data_som2, ripples_tags_som2, fs_som2)
som2_scaled = RobustScaler().fit_transform(data_som2)
canal_som2 = som2_scaled[:, 0]

datapath = "/content/Datos/Entrenamiento/Amigo2/figshare_16847521"
data_am2, fs_am2, session_name = load_data(datapath)
ripples_tags_am2 = load_ripples_tags(datapath, fs_am2)
signal_am2 = get_ripples_tags_as_signal(data_am2, ripples_tags_am2, fs_am2)
am2_scaled = RobustScaler().fit_transform(data_am2)
canal_am2 = am2_scaled[:, 0]

Obtener datos de validacion

In [ ]:
datapath = "/content/Datos/Validacion/Thy7/figshare_14960085"
data_thy7, fs_thy7, session_name = load_data(datapath)
ripples_tags_thy7 = load_ripples_tags(datapath, fs_thy7)
signal_thy7 = get_ripples_tags_as_signal(data_thy7, ripples_tags_thy7, fs_thy7)
thy7_scaled = RobustScaler().fit_transform(data_thy7)
canal_thy7 = thy7_scaled[:, 0]

datapath = "/content/Datos/Validacion/Dlx1/figshare_14959449"
data_dlx1, fs_dlx1, session_name = load_data(datapath)
ripples_tags_dlx1 = load_ripples_tags(datapath, fs_dlx1)
signal_dlx1 = get_ripples_tags_as_signal(data_dlx1, ripples_tags_dlx1, fs_dlx1)
dlx1_scaled = RobustScaler().fit_transform(data_dlx1)
canal_dlx1 = dlx1_scaled[:, 0]

Pasar las soluciones a intervalos

In [ ]:
solsIntervalo_som2 = solsToIntervals(signal_som2, inter, porcen)
solsIntervalo_am2 = solsToIntervals(signal_am2, inter, porcen)

Obtener las caracteristicas de los intervalos

In [ ]:
diffs_som2 = getPendientes(canal_som2)
pendientes_som2, amplitudes_som2, absampl_som2 = getCharacteristics(diffs_som2, canal_som2, inter)

diffs_am2 = getPendientes(canal_am2)
pendientes_am2, amplitudes_am2, absampl_am2 = getCharacteristics(diffs_am2, canal_am2, inter)

diffs_thy7 = getPendientes(canal_thy7)
pendientes_thy7, amplitudes_thy7, absampl_thy7 = getCharacteristics(diffs_thy7, canal_thy7, inter)

diffs_dlx1 = getPendientes(canal_dlx1)
pendientes_dlx1, amplitudes_dlx1, absampl_dlx1 = getCharacteristics(diffs_dlx1, canal_dlx1, inter)

Combinar datos de entrenamiento

In [ ]:
pendientes = np.concatenate((pendientes_som2, pendientes_am2))
amplitudes = np.concatenate((amplitudes_som2, amplitudes_am2))
absampl = np.concatenate((absampl_som2, absampl_am2))
solsIntervalo = np.concatenate((solsIntervalo_som2, solsIntervalo_am2))

Undersampling

In [ ]:
indexes = undersamplingIndexes(solsIntervalo)

In [ ]:
pendientes_under = pendientes[indexes]
amplitudes_under = amplitudes[indexes]
absampl_under = absampl[indexes]
solsIntervalo_under = solsIntervalo[indexes]

Obtener los datos de entrenamiento juntado las caracteristicas

In [ ]:
train = getDataSet(pendientes_under, amplitudes_under, absampl_under)

val_thy7 = getDataSet(pendientes_thy7, amplitudes_thy7, absampl_thy7)

val_dlx1 = getDataSet(pendientes_dlx1, amplitudes_dlx1, absampl_dlx1)

## XGBoost

Entrenar clasificador

In [ ]:
from xgboost import XGBClassifier

boostClassifier = XGBClassifier()
boostClassifier.fit(train, solsIntervalo_under)

Predecir soluciones

In [ ]:
boostPredict_thy7 = boostClassifier.predict(val_thy7)

Pasar predicciones a tiempos bien marcados

In [ ]:
tiemposBoostPredict_thy7 = toTimes(boostPredict_thy7, fs_thy7, zeros, inter)

Calculo F1

In [ ]:
get_score(ripples_tags_thy7, tiemposBoostPredict_thy7, threshold=0.1)

Igual para datos de DIx1

In [ ]:
boostPredict_dlx1 = boostClassifier.predict(val_dlx1)
tiemposBoostPredict_dlx1 = toTimes(boostPredict_dlx1, fs_dlx1, zeros, inter)
get_score(ripples_tags_dlx1, tiemposBoostPredict_dlx1, threshold=0.1)

## TEST

In [ ]:
datapath = "/content/Datos/Test/Test1/figshare_16856182"
data_t1, fs_t1, session_name_t1 = load_data(datapath)
t1_scaled = RobustScaler().fit_transform(data_t1)
canal_t1 = t1_scaled[:, 0]

datapath = "/content/Datos/Test/Test2/figshare_16856248"
data_t2, fs_t2, session_name_t2 = load_data(datapath)
t2_scaled = RobustScaler().fit_transform(data_t2)
canal_t2 = t2_scaled[:, 0]

In [ ]:
diffs_t1 = getPendientes(canal_t1)
pendientes_t1, amplitudes_t1, absampl_t1 = getCharacteristics(diffs_t1, canal_t1, inter)

diffs_t2 = getPendientes(canal_t2)
pendientes_t2, amplitudes_t2, absampl_t2 = getCharacteristics(diffs_t2, canal_t2, inter)

In [ ]:
test_t1 = getDataSet(pendientes_t1, amplitudes_t1, absampl_t1)

test_t2 = getDataSet(pendientes_t2, amplitudes_t2, absampl_t2)

In [ ]:
boostPredict_t1 = boostClassifier.predict(test_t1)

boostPredict_t2 = boostClassifier.predict(test_t2)

In [ ]:
tiemposBoostPredict_t1 = toTimes(boostPredict_t1, fs_t1, zeros, inter)

tiemposBoostPredict_t2 = toTimes(boostPredict_t2, fs_t2, zeros, inter)

In [ ]:
save_path_t1 = '/content/Resultados/Test1'

save_path_t2 = '/content/Resultados/Test2'

In [ ]:
write_results (save_path_t1, session_name_t1, 6, tiemposBoostPredict_t1)

write_results (save_path_t2, session_name_t2, 6, tiemposBoostPredict_t2)